In [27]:
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

In [2]:
api_key = '[API KEY]'

In [30]:
channel_ids = [
    'UC0rzsIrAxF4kCsALP6J2EsA', #Raditya Dika
    #Maybe more
]

In [31]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
)
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [36]:
def get_channel_stats(youtube, channel_ids):
    
    data_list = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )
    response = request.execute()

    # Loop through items
    for item in response['items']:
        data = {
            'channel_name': item['snippet']['title'],
            'subsrcibers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'videos': item['statistics']['videoCount'],
            'playlist_id': item['contentDetails']['relatedPlaylists']['uploads'],
        }
        data_list.append(data)
    
    return(pd.DataFrame(data_list))

In [44]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,channel_name,subsrcibers,views,videos,playlist_id
0,Raditya Dika,10100000,2201576441,1974,UU0rzsIrAxF4kCsALP6J2EsA


In [49]:
playlist_id = 'UU0rzsIrAxF4kCsALP6J2EsA'

def get_video_ids(youtube, playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet, contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')

    return video_ids

In [63]:
video_ids = get_video_ids(youtube, playlist_id)

len(video_ids)

1973

In [57]:
def get_video_details(youtube, video_ids):
    all_video_info = []

    request = youtube.videos().list(
        part="snippet, contentDetails, statistics",
        id=video_ids[0:3],
    )
    response = request.execute()

    for video in response['items']:
        keep_stats = {
            'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
            'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
            'contentDetails': ['duration', 'definition', 'caption'],
        }

        video_info = {}
        video_info['video_id'] = video['id']

        for k in keep_stats.keys():
            for v in keep_stats[k]:
                try:
                    video_info[v] = video[k][v]
                except:
                    video_info[v] = None

        all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [58]:
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,L7tUnDWwqvQ,Raditya Dika,Makanan Favorit Keluarga Istri Saya!,"Beberapa waktu lalu, gue ngundang tante gue un...",[raditya dika],2023-12-29T13:25:30Z,126930,3840,None,302,PT15M25S,hd,false
1,vIoaAfglUlA,Raditya Dika,Setelah keluar kata2 sakti baru deh nurut..,,None,2023-12-25T03:42:53Z,40769,1495,None,35,PT25S,hd,false
2,fodkqroWBpY,Raditya Dika,Hohohihihohohihihohohihi,Beli video kompilasi tur Stand Up Comedy Radit...,[raditya dika],2023-12-22T12:00:34Z,256938,5853,None,311,PT33M43S,hd,false


In [67]:
def get_videos_comments(youtube, video_ids):
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments)     

In [68]:
comments_df = get_videos_comments(youtube, video_ids)
comments_df

Could not get comments for video xilMLXvl76w
Could not get comments for video FXrNRyHqRVI
